# NFL Scores and Betting Data Cleaning

In [60]:
# Dependencies and Setup
import pandas as pd

In [61]:
# stadium csv has an encoding issue, will resolve to fix but data is not vital - skipping for now
# stadiums_df = pd.read_csv("raw data/nfl-scores-and-betting-data/nfl_stadiums.csv") 
teams_df = pd.read_csv("raw data/nfl-scores-and-betting-data/nfl_teams.csv")
scorelines_df = pd.read_csv("raw data/nfl-scores-and-betting-data/spreadspoke_scores.csv")

In [62]:
scorelines_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/02/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,09/03/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,09/04/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,09/09/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,09/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN


In [63]:
scorelines_df.tail()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
12673,01/12/2020,2019,Division,True,Green Bay Packers,28,23,Seattle Seahawks,GB,-4.5,45.5,Lambeau Field,False,NaN,NaN,NaN,NaN
12674,01/12/2020,2019,Division,True,Kansas City Chiefs,51,31,Houston Texans,KC,-10.0,50.5,Arrowhead Stadium,False,NaN,NaN,NaN,NaN
12675,01/19/2020,2019,Conference,True,Kansas City Chiefs,35,24,Tennessee Titans,KC,-7.0,51,Arrowhead Stadium,False,NaN,NaN,NaN,NaN
12676,01/19/2020,2019,Conference,True,San Francisco 49ers,37,20,Green Bay Packers,SF,-8.0,46.5,Levi's Stadium,False,NaN,NaN,NaN,NaN
12677,02/02/2020,2019,Superbowl,True,Kansas City Chiefs,31,20,San Francisco 49ers,KC,-1.5,53,Hard Rock Stadium,True,NaN,NaN,NaN,NaN


In [64]:
scorelines_trimmed_df = scorelines_df[scorelines_df['schedule_season'] >= 2000]

In [65]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
7354,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,ATL,-6.5,46.5,Georgia Dome,False,72.0,0.0,NaN,DOME
7355,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,BUF,-1.0,40,Ralph Wilson Stadium,False,70.0,7.0,82,NaN
7356,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,JAX,-10.5,38.5,FirstEnergy Stadium,False,75.0,8.0,86,NaN
7357,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,DAL,-6.0,39.5,Texas Stadium,False,95.0,7.0,32,NaN
7358,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,GB,-2.5,44,Lambeau Field,False,69.0,13.0,87,NaN


In [66]:
scorelines_trimmed_df.reset_index(drop=True, inplace=True)

In [67]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,ATL,-6.5,46.5,Georgia Dome,False,72.0,0.0,NaN,DOME
1,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,BUF,-1.0,40,Ralph Wilson Stadium,False,70.0,7.0,82,NaN
2,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,JAX,-10.5,38.5,FirstEnergy Stadium,False,75.0,8.0,86,NaN
3,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,DAL,-6.0,39.5,Texas Stadium,False,95.0,7.0,32,NaN
4,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,GB,-2.5,44,Lambeau Field,False,69.0,13.0,87,NaN


In [68]:
scorelines_trimmed_df.drop(['weather_temperature', 'weather_wind_mph', 'weather_humidity', 'weather_detail'], axis=1, inplace=True)

In [69]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral
0,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,ATL,-6.5,46.5,Georgia Dome,False
1,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,BUF,-1.0,40,Ralph Wilson Stadium,False
2,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,JAX,-10.5,38.5,FirstEnergy Stadium,False
3,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,DAL,-6.0,39.5,Texas Stadium,False
4,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,GB,-2.5,44,Lambeau Field,False


In [70]:
scorelines_trimmed_df['team_favorite_id'].unique()

array(['ATL', 'BUF', 'JAX', 'DAL', 'GB', 'IND', 'MIA', 'MIN', 'TB',
       'PICK', 'NYG', 'OAK', 'BAL', 'WAS', 'LAR', 'ARI', 'CIN', 'DEN',
       'PHI', 'LAC', 'CAR', 'TEN', 'NYJ', 'CHI', 'PIT', 'KC', 'NE', 'SEA',
       'NO', 'DET', 'SF', 'CLE', 'HOU'], dtype=object)

In [71]:
scorelines_trimmed_df['team_home'].unique()

array(['Atlanta Falcons', 'Buffalo Bills', 'Cleveland Browns',
       'Dallas Cowboys', 'Green Bay Packers', 'Kansas City Chiefs',
       'Miami Dolphins', 'Minnesota Vikings', 'New England Patriots',
       'New Orleans Saints', 'New York Giants', 'Oakland Raiders',
       'Pittsburgh Steelers', 'Washington Redskins', 'St. Louis Rams',
       'Arizona Cardinals', 'Baltimore Ravens', 'Cincinnati Bengals',
       'Denver Broncos', 'Detroit Lions', 'Indianapolis Colts',
       'Philadelphia Eagles', 'San Diego Chargers', 'San Francisco 49ers',
       'Seattle Seahawks', 'Tampa Bay Buccaneers', 'Tennessee Titans',
       'New York Jets', 'Carolina Panthers', 'Chicago Bears',
       'Jacksonville Jaguars', 'Houston Texans', 'Los Angeles Rams',
       'Los Angeles Chargers'], dtype=object)

In [72]:
team_names = {'ATL': 'Atlanta Falcons', 'BUF': 'Buffalo Bills', 'JAX': 'Jacksonville Jaguars', 'DAL': 'Dallas Cowboys', 'GB': 'Green Bay Packers', 'IND': 'Indianapolis Colts', 'MIA': 'Miami Dolphins', 'MIN': 'Minnesota Vikings', 'TB': 'Tampa Bay Buccaneers', 'NYG': 'New York Giants', 'OAK': 'Oakland Raiders', 'BAL': 'Baltimore Ravens', 'ARI': 'Arizona Cardinals', 'CIN': 'Cincinnati Bengals', 'DEN': 'Denver Broncos', 'PHI': 'Philidelphia Eagles', 'LAC': 'Los Angeles Chargers', 'CAR': 'Carolina Panthers', 'TEN': 'Tennessee Titans', 'NYJ': 'New York Jets', 'CHI': 'Chicago Bears', 'PIT': 'Pittsburgh Steelers', 'KC': 'Kansas City Chiefs', 'NE': 'New England Patriots', 'SEA': 'Seattle Seahawks', 'NO': 'New Orleans Saints', 'DET': 'Detroit Lions', 'SF': 'San Francisco 49ers', 'CLE': 'Cleveland Browns', 'HOU': 'Houston Texans', 'WAS': 'Washington Redskins', 'LAR': 'Los Angeles Rams'}

In [73]:
scorelines_trimmed_df.replace({"team_favorite_id": team_names}, inplace=True) 

In [74]:
scorelines_trimmed_df['team_favorite_id'].unique()

array(['Atlanta Falcons', 'Buffalo Bills', 'Jacksonville Jaguars',
       'Dallas Cowboys', 'Green Bay Packers', 'Indianapolis Colts',
       'Miami Dolphins', 'Minnesota Vikings', 'Tampa Bay Buccaneers',
       'PICK', 'New York Giants', 'Oakland Raiders', 'Baltimore Ravens',
       'Washington Redskins', 'Los Angeles Rams', 'Arizona Cardinals',
       'Cincinnati Bengals', 'Denver Broncos', 'Philidelphia Eagles',
       'Los Angeles Chargers', 'Carolina Panthers', 'Tennessee Titans',
       'New York Jets', 'Chicago Bears', 'Pittsburgh Steelers',
       'Kansas City Chiefs', 'New England Patriots', 'Seattle Seahawks',
       'New Orleans Saints', 'Detroit Lions', 'San Francisco 49ers',
       'Cleveland Browns', 'Houston Texans'], dtype=object)

In [75]:
scorelines_trimmed_df['team_win'] = ""
scorelines_trimmed_df['home_away_win'] = ""
scorelines_trimmed_df['fav_win'] = ""
scorelines_trimmed_df['over_under_result'] = ""
scorelines_trimmed_df['fav_spread_cover'] = ""

In [76]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,team_win,home_away_win,fav_win,over_under_result,fav_spread_cover
0,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,Atlanta Falcons,-6.5,46.5,Georgia Dome,False,,,,,
1,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,Buffalo Bills,-1.0,40,Ralph Wilson Stadium,False,,,,,
2,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,Jacksonville Jaguars,-10.5,38.5,FirstEnergy Stadium,False,,,,,
3,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,Dallas Cowboys,-6.0,39.5,Texas Stadium,False,,,,,
4,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,Green Bay Packers,-2.5,44,Lambeau Field,False,,,,,


In [83]:
for index, row in scorelines_trimmed_df.iterrows():

    if scorelines_trimmed_df.loc[index, 'score_home'] > scorelines_trimmed_df.loc[index, 'score_away']:
        scorelines_trimmed_df.loc[index, 'team_win'] = scorelines_trimmed_df.loc[index, 'team_home']
        scorelines_trimmed_df.loc[index, 'home_away_win'] = "Home"
    else:
        scorelines_trimmed_df.loc[index, 'team_win'] = scorelines_trimmed_df.loc[index, 'team_away']
        scorelines_trimmed_df.loc[index, 'home_away_win'] = "Away"

    if scorelines_trimmed_df.loc[index, 'team_win'] == scorelines_trimmed_df.loc[index, 'team_favorite_id']:
        scorelines_trimmed_df.loc[index, 'fav_win'] = "True"
    else:
        scorelines_trimmed_df.loc[index, 'fav_win'] = "False"

    if (scorelines_trimmed_df.loc[index, 'score_home'] + scorelines_trimmed_df.loc[index, 'score_away']) > float(scorelines_trimmed_df.loc[index, 'over_under_line']):
        scorelines_trimmed_df.loc[index, 'over_under_result'] = "Over"
    elif (scorelines_trimmed_df.loc[index, 'score_home'] + scorelines_trimmed_df.loc[index, 'score_away']) == scorelines_trimmed_df.loc[index, 'over_under_line']:
        scorelines_trimmed_df.loc[index, 'over_under_result'] = "Push"
    else:
        scorelines_trimmed_df.loc[index, 'over_under_result'] = "Under"

    if scorelines_trimmed_df.loc[index, 'fav_win'] == 'True':
        if abs(scorelines_trimmed_df.loc[index, 'score_home'] - scorelines_trimmed_df.loc[index, 'score_away']) > abs(float(scorelines_trimmed_df.loc[index, 'spread_favorite'])):
            scorelines_trimmed_df.loc[index, 'fav_spread_cover'] = "True"
        else:
            scorelines_trimmed_df.loc[index, 'fav_spread_cover'] = "False"
    else:
        scorelines_trimmed_df.loc[index, 'fav_spread_cover'] = "False"

In [84]:
scorelines_trimmed_df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,team_win,home_away_win,fav_win,over_under_result,fav_spread_cover
0,09/03/2000,2000,1,False,Atlanta Falcons,36,28,San Francisco 49ers,Atlanta Falcons,-6.5,46.5,Georgia Dome,False,Atlanta Falcons,Home,True,Over,True
1,09/03/2000,2000,1,False,Buffalo Bills,16,13,Tennessee Titans,Buffalo Bills,-1.0,40,Ralph Wilson Stadium,False,Buffalo Bills,Home,True,Under,True
2,09/03/2000,2000,1,False,Cleveland Browns,7,27,Jacksonville Jaguars,Jacksonville Jaguars,-10.5,38.5,FirstEnergy Stadium,False,Jacksonville Jaguars,Away,True,Under,True
3,09/03/2000,2000,1,False,Dallas Cowboys,14,41,Philadelphia Eagles,Dallas Cowboys,-6.0,39.5,Texas Stadium,False,Philadelphia Eagles,Away,False,Over,False
4,09/03/2000,2000,1,False,Green Bay Packers,16,20,New York Jets,Green Bay Packers,-2.5,44,Lambeau Field,False,New York Jets,Away,False,Under,False


In [85]:
underdog_wins = {}

In [86]:
for index, row in scorelines_trimmed_df.iterrows():
    
    if scorelines_trimmed_df.loc[index, 'fav_win'] == "False":
        if scorelines_trimmed_df.loc[index, 'team_win'] not in underdog_wins:
            underdog_wins[scorelines_trimmed_df.loc[index, 'team_win']] = 1
        else:
            underdog_wins[scorelines_trimmed_df.loc[index, 'team_win']] += 1

In [87]:
underdog_wins

{'Philadelphia Eagles': 205,
 'New York Jets': 65,
 'Detroit Lions': 52,
 'St. Louis Rams': 111,
 'Cleveland Browns': 57,
 'Oakland Raiders': 64,
 'New York Giants': 57,
 'New Orleans Saints': 53,
 'Atlanta Falcons': 58,
 'Miami Dolphins': 75,
 'Minnesota Vikings': 57,
 'Denver Broncos': 51,
 'Dallas Cowboys': 51,
 'Green Bay Packers': 45,
 'San Francisco 49ers': 54,
 'Kansas City Chiefs': 56,
 'Washington Redskins': 74,
 'Indianapolis Colts': 44,
 'New England Patriots': 41,
 'Chicago Bears': 62,
 'Pittsburgh Steelers': 51,
 'Baltimore Ravens': 52,
 'Tennessee Titans': 62,
 'Cincinnati Bengals': 62,
 'Jacksonville Jaguars': 67,
 'Arizona Cardinals': 69,
 'Buffalo Bills': 62,
 'Carolina Panthers': 62,
 'Seattle Seahawks': 53,
 'San Diego Chargers': 140,
 'Tampa Bay Buccaneers': 59,
 'Houston Texans': 52,
 'Los Angeles Rams': 7,
 'Los Angeles Chargers': 10}

In [88]:
scorelines_trimmed_df.to_csv("output data/spreadspoke_scores_trimmed.csv", index=False)